Read data from all csv files

In [168]:
import pandas as pd
import numpy as np
import os

folder_path = 'C:\GUVI Projects\Sales Analytics - Amazon India\Data'
df_sales = pd.DataFrame()

for yr in range(2015, 2026):
    filename = f'amazon_india_{yr}.csv'
    full_path = os.path.join(folder_path, filename)

    if len(df_sales) == 0:
        df_sales = pd.read_csv(full_path)
    else:
        df = pd.read_csv(full_path)
        df_sales = pd.concat([df_sales, df], ignore_index=True)

df_sales.info()

<>:5: SyntaxWarning: "\G" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\G"? A raw string is also an option.
<>:5: SyntaxWarning: "\G" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\G"? A raw string is also an option.
C:\Users\raagh\AppData\Local\Temp\ipykernel_4904\1854143993.py:5: SyntaxWarning: "\G" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\G"? A raw string is also an option.
  folder_path = 'C:\GUVI Projects\Sales Analytics - Amazon India\Data'


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1127609 entries, 0 to 1127608
Data columns (total 34 columns):
 #   Column                  Non-Null Count    Dtype  
---  ------                  --------------    -----  
 0   transaction_id          1127609 non-null  object 
 1   order_date              1127609 non-null  object 
 2   customer_id             1127609 non-null  object 
 3   product_id              1127609 non-null  object 
 4   product_name            1127609 non-null  object 
 5   category                1127609 non-null  object 
 6   subcategory             1127609 non-null  object 
 7   brand                   1127609 non-null  object 
 8   original_price_inr      1127609 non-null  object 
 9   discount_percent        1127609 non-null  float64
 10  discounted_price_inr    1127609 non-null  float64
 11  quantity                1127609 non-null  int64  
 12  subtotal_inr            1127609 non-null  float64
 13  delivery_charges        1037408 non-null  float64
 14  fi

In [169]:
df_sales[['order_date','order_month','order_year']].value_counts()

order_date  order_month  order_year
2020-12-02  12           2020          543
2020-12-05  12           2020          528
2020-12-26  12           2020          525
2020-12-14  12           2020          519
2020-12-18  12           2020          518
                                      ... 
02/16/2019  2            2019            1
02/18/2016  2            2016            1
01/02/2018  1            2018            1
31-03-2015  3            2015            1
17/05/2015  5            2015            1
Name: count, Length: 15841, dtype: int64

In [170]:
def parse_date(raw_date, month, year):
    try:
        # Extract day
        if '/' in raw_date:
            part1, part2, part3 = raw_date.split('/')
        elif '-' in raw_date:
            part1, part2, part3 = raw_date.split('-')
        else:
            return pd.NaT

        if len(part1) == 4:
            if int(part2) == month:
                day = int(part3)
            else:
                day = int(part2)
        else:
            if int(part2) == month:
                day = int(part1)
            else:
                day = int(part2)

        # Build final date using passed month & year
        return pd.to_datetime(
            f"{int(year)}-{int(month)}-{day}",
            errors='coerce'
        )

    except:
        return pd.NaT

In [171]:
parse_date('09-10-2025', 10, 2025)

Timestamp('2025-10-09 00:00:00')

In [172]:
# date conversion
#df_sales['order_date_cleaned'] = pd.to_datetime(df_sales['order_date'],format='mixed', dayfirst=True, errors='coerce')
#df_sales[df_sales['order_date_cleaned'].dt.month == df_sales['order_month']][['order_date_cleaned','order_date','order_month','order_year']]
df_sales['order_date_cleaned'] = df_sales.apply(
    lambda row: parse_date(row['order_date'], row['order_month'], row['order_year']),
    axis=1
)


In [173]:
# date conversion
#df_sales['order_date_cleaned'] = pd.to_datetime(df_sales['order_date'],format='mixed', dayfirst=True, errors='coerce')
cond = df_sales['order_date_cleaned'].dt.month != df_sales['order_month']
df_sales[cond][['order_date_cleaned','order_date','order_month','order_year']]

#df_sales[cond]['order_date_cleaned'] = pd.to_datetime(df_sales[cond]['order_date_cleaned'],format='MM/DD/YYYY', dayfirst=False, errors='coerce')


Empty DataFrame
Columns: [order_date_cleaned, order_date, order_month, order_year]
Index: []

In [174]:
#Check for nulls
df_sales.isnull().sum()

transaction_id                 0
order_date                     0
customer_id                    0
product_id                     0
product_name                   0
category                       0
subcategory                    0
brand                          0
original_price_inr             0
discount_percent               0
discounted_price_inr           0
quantity                       0
subtotal_inr                   0
delivery_charges           90201
final_amount_inr               0
customer_city                  0
customer_state                 0
customer_tier                  0
customer_spending_tier         0
customer_age_group        135315
payment_method                 0
delivery_days                  0
delivery_type                  0
is_prime_member                0
is_festival_sale               0
festival_name             777736
customer_rating           341696
return_status                  0
order_month                    0
order_year                     0
order_quar

In [175]:
# numerical fileds
df_sales['original_price_inr'].value_counts()
df_sales['original_price_inr'] = df_sales['original_price_inr'].str.replace(r"Rs\s+",'',regex=True)
df_sales['original_price_inr'] = df_sales['original_price_inr'].str.replace(r"₹",'')
df_sales['original_price_inr'] = df_sales['original_price_inr'].str.replace(r",",'')
df_sales['original_price_inr'].str.isspace().sum()

df_sales['original_price_inr'] = pd.to_numeric(df_sales['original_price_inr'], errors='raise')
df_sales['original_price_inr'] = df_sales['original_price_inr'].abs()

In [176]:
df_sales['customer_age_group'].value_counts()

customer_age_group
26-35    347899
18-25    316424
36-45    199100
46-55     99012
55+       29859
Name: count, dtype: int64

In [177]:
# df_sales[df_sales['return_status'] == 'Returned']
df_sales['customer_rating'].value_counts()

customer_rating
4.5          245857
5.0          185749
4.0          174681
3.5           76153
3.0           40617
4/5           10914
4.5 stars      8618
5.0 stars      6368
4.0 stars      6069
5/5            4787
4.5/5.0        4230
4              3677
5.0/5.0        3361
3/5            3079
4.0/5.0        2955
3.5 stars      2695
5              1587
3.0 stars      1431
3.5/5.0        1309
3              1048
3.0/5.0         728
Name: count, dtype: int64

In [178]:
df_sales['customer_rating'] = df_sales['customer_rating'].str.replace(r"/5.0|/5| stars","", regex=True)
df_sales['customer_rating'] = pd.to_numeric(df_sales['customer_rating']).round(1)
df_sales['customer_rating'].value_counts()


customer_rating
4.5    258705
5.0    201852
4.0    198296
3.5     80157
3.0     46903
Name: count, dtype: int64

In [179]:
df_sales['festival_name'].value_counts()

festival_name
Back to School                  85396
Diwali Sale                     83611
Amazon Great Indian Festival    53062
Summer Sale                     44931
Holi Festival                   39608
Republic Day Sale               20183
Valentine Sale                  14106
Prime Day                        8976
Name: count, dtype: int64

In [180]:
df_sales['return_status'].value_counts()

return_status
Delivered    1022431
Returned       79125
Cancelled      26053
Name: count, dtype: int64

In [181]:
df_sales['customer_spending_tier'].value_counts()

customer_spending_tier
Standard    688120
Budget      232696
Premium     206793
Name: count, dtype: int64

In [182]:
df_sales['customer_state'].value_counts()

customer_state
Maharashtra       246558
Delhi             123469
Uttar Pradesh     118403
Gujarat           112339
Tamil Nadu        111448
Karnataka         102523
West Bengal        66867
Telangana          44573
Rajasthan          40453
Punjab             38466
Madhya Pradesh     33592
Kerala             24940
Andhra Pradesh     22167
Bihar              21339
Odisha             20472
Name: count, dtype: int64

In [183]:
df_sales['customer_city'].value_counts()

customer_city
Mumbai           140536
Delhi            122365
Bangalore        101416
Chennai           83646
Pune              67703
Kolkata           65767
Ahmedabad         50861
Hyderabad         44573
Jaipur            40453
Surat             40400
Nagpur            37211
Kanpur            34334
Lucknow           34077
Indore            33592
Coimbatore        26695
Kochi             24940
Visakhapatnam     22167
Patna             21339
Vadodara          21078
Bhubaneswar       20472
Chandigarh        19575
Ludhiana          18891
Saharanpur         6705
Meerut             6641
Bareilly           6590
Aligarh            6407
Allahabad          6030
Varanasi           5897
Gorakhpur          5867
Moradabad          5855
Mumbai              304
New Delhi           304
Kolkata             294
Bombay              289
Banglore            287
CHENNAI             285
KOLKATA             285
delhi               283
Delhi NCR           282
chenai              280
mumba               279
BA

In [184]:

df_sales['customer_city'] = df_sales['customer_city'].str.replace(r"Bangalore|Banglore|BANGALORE|Bengalore","Bengaluru",regex=True)
df_sales['customer_city'] = df_sales['customer_city'].str.replace(r"Mumbai\s+|Bombay|mumba|MUMBAI","Mumbai",regex=True)
df_sales['customer_city'] = df_sales['customer_city'].str.replace(r"Kolkata\s+|KOLKATA|Calcutta|kolkata","Kolkata",regex=True)
df_sales['customer_city'] = df_sales['customer_city'].str.replace(r"CHENNAI|chenai|Chennai\s+|Madras","Chennai",regex=True)
df_sales['customer_city'] = df_sales['customer_city'].str.replace(r"Delhi NCR|New Delhi|Delhi|delhi|DELHI","New Delhi",regex=True)


In [185]:
df_sales['customer_city'].value_counts()

customer_city
Mumbai           141644
New Delhi        123469
Bengaluru        102523
Chennai           84753
Pune              67703
Kolkata           66867
Ahmedabad         50861
Hyderabad         44573
Jaipur            40453
Surat             40400
Nagpur            37211
Kanpur            34334
Lucknow           34077
Indore            33592
Coimbatore        26695
Kochi             24940
Visakhapatnam     22167
Patna             21339
Vadodara          21078
Bhubaneswar       20472
Chandigarh        19575
Ludhiana          18891
Saharanpur         6705
Meerut             6641
Bareilly           6590
Aligarh            6407
Allahabad          6030
Varanasi           5897
Gorakhpur          5867
Moradabad          5855
Name: count, dtype: int64

In [186]:
df_sales['delivery_charges'].value_counts()

delivery_charges
0.0     1037400
40.0          8
Name: count, dtype: int64

In [187]:
df_sales.columns

Index(['transaction_id', 'order_date', 'customer_id', 'product_id',
       'product_name', 'category', 'subcategory', 'brand',
       'original_price_inr', 'discount_percent', 'discounted_price_inr',
       'quantity', 'subtotal_inr', 'delivery_charges', 'final_amount_inr',
       'customer_city', 'customer_state', 'customer_tier',
       'customer_spending_tier', 'customer_age_group', 'payment_method',
       'delivery_days', 'delivery_type', 'is_prime_member', 'is_festival_sale',
       'festival_name', 'customer_rating', 'return_status', 'order_month',
       'order_year', 'order_quarter', 'product_weight_kg', 'is_prime_eligible',
       'product_rating', 'order_date_cleaned'],
      dtype='object')

In [188]:
df_sales['is_prime_eligible'].value_counts()

is_prime_eligible
True     840778
False    185642
1         27650
Yes       27631
TRUE      27607
FALSE      6183
No         6079
0          6039
Name: count, dtype: int64

In [189]:
df_sales['is_prime_eligible'].replace({'Yes': True, '1': True, 'TRUE': True}, inplace=True)
df_sales['is_prime_eligible'].replace({'No': False, '0': False, 'FALSE': False}, inplace=True)
df_sales['is_prime_eligible'] = df_sales['is_prime_eligible'].astype(bool)
df_sales['is_prime_eligible'].value_counts()


is_prime_eligible
True     1109308
False      18301
Name: count, dtype: int64

In [190]:
df_sales['is_festival_sale'].value_counts()


is_festival_sale
False    707951
True     318446
0         23568
FALSE     23150
No        23067
Yes       10518
TRUE      10488
1         10421
Name: count, dtype: int64

In [191]:
df_sales['is_festival_sale'].replace({'0':False, '1': True, 'Yes': True, 'No': False, 'TRUE': True, 'FALSE': False}, inplace=True)
df_sales['is_festival_sale'] = df_sales['is_festival_sale'].astype(bool)
df_sales['is_festival_sale'].value_counts()

is_festival_sale
True     1057824
False      69785
Name: count, dtype: int64

In [192]:
df_sales['is_prime_member'].value_counts()

is_prime_member
False    635593
True     390675
FALSE     21086
0         21078
No        20728
Yes       12894
1         12871
TRUE      12684
Name: count, dtype: int64

In [193]:
df_sales['is_prime_member'].replace({'0':False, '1': True, 'Yes': True, 'No': False, 'TRUE': True, 'FALSE': False}, inplace=True)
df_sales['is_prime_member'] = df_sales['is_prime_member'].astype(bool)
df_sales['is_prime_member'].value_counts()

is_prime_member
True     1064717
False      62892
Name: count, dtype: int64

In [194]:
df_sales['category'].value_counts()

category
Electronics                  1126726
Electronic                       229
ELECTRONICS                      225
Electronics & Accessories        218
Electronicss                     211
Name: count, dtype: int64

In [195]:
df_sales['category'].replace({'Electronic':'Electronics', 'ELECTRONICS':'Electronics', 'Electronics & Accessories':'Electronics', 'Electronicss':'Electronics'}, inplace=True)
df_sales['category'].value_counts()

category
Electronics    1127609
Name: count, dtype: int64

In [196]:
df_sales['delivery_days'].value_counts()

delivery_days
3           288488
1           210462
4           206438
5           136608
2           126280
6           102668
7            34116
-1            6836
1-2 days      4534
Same Day      4469
Express       2317
0             2218
15            2175
Name: count, dtype: int64

In [197]:
# Considering max days
df_sales['delivery_days'].replace({'1-2 days':2, 'Same Day':0, '-1':0, 'Express': 1}, inplace=True)
df_sales['delivery_days'] = df_sales['delivery_days'].astype(int)
df_sales['delivery_days'].value_counts()

delivery_days
3     288488
1     212779
4     206438
5     136608
2     130814
6     102668
7      34116
0      13523
15      2175
Name: count, dtype: int64

In [198]:
cols = ['original_price_inr', 'discount_percent', 'discounted_price_inr','quantity', 'subtotal_inr', 'delivery_charges', 'final_amount_inr']

df_sales[cols].head()

original_price_inr  discount_percent  discounted_price_inr  quantity  \
0           123614.29             27.91              89112.17         1   
1            54731.86              0.00              54731.86         1   
2            97644.25             46.93              51818.26         2   
3            21947.26              0.00              21947.26         1   
4            54731.86              0.00              54731.86         2   

   subtotal_inr  delivery_charges  final_amount_inr  
0      89112.17               0.0          89112.17  
1      54731.86               0.0          54731.86  
2     103636.52               NaN         103636.52  
3      21947.26               0.0          21947.26  
4     109463.72               0.0         109463.72

In [199]:
df_sales['original_price_inr'].value_counts()

original_price_inr
45363.47       433
52280.35       413
54731.86       401
46234.09       391
46458.94       386
              ... 
21444.00         1
10330.00         1
12546124.00      1
771294.50        1
35256.00         1
Name: count, Length: 25178, dtype: int64

In [200]:
df_sales['calc_discounted_price'] = (df_sales['original_price_inr'] * ((100 - df_sales['discount_percent'])/100)).round(2)
condition1 = (df_sales['calc_discounted_price'] >= (df_sales['discounted_price_inr'] -20)) & (df_sales['calc_discounted_price'] <= (df_sales['discounted_price_inr'] + 20)) 
col1 = ['product_name','calc_discounted_price','original_price_inr', 'discount_percent', 'discounted_price_inr']
df_sales[~condition1][col1]

product_name  calc_discounted_price  \
416          Samsung Galaxy J7 16GB Blue              268040.96   
445           OnePlus Pad 4GB RAM Silver             3901009.00   
1194     Xiaomi Redmi Note 4G 64GB Black             2631710.00   
1319           Xiaomi Redmi 2 16GB White             4376740.00   
1434                      Fitbit Tracker              180962.50   
...                                  ...                    ...   
1125906                Oppo F5 16GB Blue              238419.80   
1126106  Samsung Galaxy S21+ 128GB Black              776207.60   
1126272       Lenovo Slate 8GB RAM Black              998871.88   
1126343        Xiaomi Poco F7 64GB Black              185752.90   
1127194          Xiaomi Mi A1 16GB Black             1998507.65   

         original_price_inr  discount_percent  discounted_price_inr  
416                448304.0             40.21              26803.39  
445               3901009.0              0.00              39010.09  
1194              2631710.0              0.00              26317.10  
1319              4376740.0              0.00              43767.40  
1434               198641.6              8.90              18096.64  
...                     ...               ...                   ...  
1125906            238419.8              0.00              23841.98  
1126106            776207.6              0.00              77620.76  
1126272           1111215.8             10.11              99885.67  
1126343            185752.9              0.00              18575.29  
1127194           2547817.0             21.56              19985.48  

[5373 rows x 5 columns]

In [201]:
# 
df_sales['calc_original_price'] = np.where(~condition1, ((df_sales['discounted_price_inr']*100)/(100-df_sales['discount_percent'])).round(2), df_sales['original_price_inr'])

df_sales['calc_discounted_price2'] = (df_sales['calc_original_price'] * ((100 - df_sales['discount_percent'])/100)).round(2)

condition2 = (df_sales['calc_discounted_price2'] >= (df_sales['discounted_price_inr'] -20)) & (df_sales['calc_discounted_price2'] <= (df_sales['discounted_price_inr'] + 20)) 
col2 = ['product_name','calc_discounted_price2','calc_original_price','original_price_inr', 'discount_percent', 'discounted_price_inr']
df_sales[~condition2][col2]

Empty DataFrame
Columns: [product_name, calc_discounted_price2, calc_original_price, original_price_inr, discount_percent, discounted_price_inr]
Index: []

In [202]:
condition3 = (df_sales['discount_percent'] == 0) & (df_sales['calc_discounted_price'] <= (df_sales['discounted_price_inr'] - 5)) & (df_sales['calc_discounted_price'] >= (df_sales['discounted_price_inr'] + 5))
df_sales[condition3][col1]


Empty DataFrame
Columns: [product_name, calc_discounted_price, original_price_inr, discount_percent, discounted_price_inr]
Index: []

In [203]:
dup_df = df_sales.groupby(['order_date', 'customer_id', 'product_id','calc_original_price']).filter(lambda x: len(x) > 1)
dup_df.sort_values(['order_date', 'customer_id', 'product_id'])

transaction_id  order_date         customer_id   product_id  \
220850      TXN_2018_00055026  01-08-2018  CUST_2018_00028644  PROD_001923   
265129  TXN_2018_00055026_DUP  01-08-2018  CUST_2018_00028644  PROD_001923   
464655      TXN_2020_00077731  01-08-2020  CUST_2020_00035605  PROD_000054   
530001  TXN_2020_00077731_DUP  01-08-2020  CUST_2020_00035605  PROD_000054   
365800      TXN_2019_00100481  01-11-2019  CUST_2019_00037498  PROD_000105   
...                       ...         ...                 ...          ...   
264909  TXN_2018_00068017_DUP  31-10-2018  CUST_2016_00018651  PROD_000272   
384149      TXN_2019_00118830  31-12-2019  CUST_2018_00018479  PROD_000159   
386915  TXN_2019_00118830_DUP  31-12-2019  CUST_2018_00018479  PROD_000159   
329545      TXN_2019_00064226  31/08/2019  CUST_2019_00039742  PROD_000456   
386613  TXN_2019_00064226_DUP  31/08/2019  CUST_2019_00039742  PROD_000456   

                                product_name     category  subcategory  \
220850                  Fitbit Watch Premium  Electronics  Smart Watch   
265129                  Fitbit Watch Premium  Electronics  Smart Watch   
464655          OnePlus OnePlus 2 64GB Black  Electronics  Smartphones   
530001          OnePlus OnePlus 2 64GB Black  Electronics  Smartphones   
365800  Motorola Moto E (2nd Gen) 64GB White  Electronics  Smartphones   
...                                      ...          ...          ...   
264909         OnePlus OnePlus 5T 64GB Black  Electronics  Smartphones   
384149          OnePlus OnePlus 3 32GB White  Electronics  Smartphones   
386915          OnePlus OnePlus 3 32GB White  Electronics  Smartphones   
329545                   Vivo Y95 64GB White  Electronics  Smartphones   
386613                   Vivo Y95 64GB White  Electronics  Smartphones   

           brand  original_price_inr  discount_percent  ...  order_month  \
220850    Fitbit            52511.57              0.00  ...            8   
265129    Fitbit            52511.57              0.00  ...            8   
464655   OnePlus            76864.44             10.61  ...            8   
530001   OnePlus            76864.44             10.61  ...            8   
365800  Motorola            22751.46             24.72  ...           11   
...          ...                 ...               ...  ...          ...   
264909   OnePlus            89305.36             50.09  ...           10   
384149   OnePlus            92223.55             13.89  ...           12   
386915   OnePlus            92223.55             13.89  ...           12   
329545      Vivo            20054.30              0.00  ...            8   
386613      Vivo            20054.30              0.00  ...            8   

        order_year  order_quarter  product_weight_kg  is_prime_eligible  \
220850        2018              3               0.06               True   
265129        2018              3               0.06               True   
464655        2020              3               0.21               True   
530001        2020              3               0.21               True   
365800        2019              4               0.21               True   
...            ...            ...                ...                ...   
264909        2018              4               0.18               True   
384149        2019              4               0.20               True   
386915        2019              4               0.20               True   
329545        2019              3               0.21               True   
386613        2019              3               0.21               True   

       product_rating order_date_cleaned calc_discounted_price  \
220850            4.8         2018-08-01              52511.57   
265129            4.8         2018-08-01              52511.57   
464655            4.2         2020-08-01              68709.12   
530001            4.2         2020-08-01              68709.12   
365800            3.8         2019-11-01     

In [204]:
df_sales.drop_duplicates(subset=['order_date', 'customer_id', 'product_id','calc_original_price'], keep='first', inplace=True)
df_sales['transaction_id'] = df_sales['transaction_id'].str.replace('_DUP','')
df_sales

transaction_id  order_date         customer_id   product_id  \
0        TXN_2015_00000001  2015-01-25  CUST_2015_00003884  PROD_000021   
1        TXN_2015_00000002  2015-01-05  CUST_2015_00011709  PROD_000055   
2        TXN_2015_00000003  2015-01-24  CUST_2015_00004782  PROD_000039   
3        TXN_2015_00000004  2015-01-28  CUST_2015_00008105  PROD_000085   
4        TXN_2015_00000005  2015-01-31  CUST_2015_00002955  PROD_000055   
...                    ...         ...                 ...          ...   
1127222  TXN_2025_00076999  2025-12-12  CUST_2025_00019394  PROD_001739   
1127223  TXN_2025_00077000  2025-12-24  CUST_2021_00014781  PROD_000133   
1127325  TXN_2025_00022234  2025-04-27  CUST_2015_00004317  PROD_000771   
1127494  TXN_2025_00067929  2025-12-13  CUST_2024_00037053  PROD_000388   
1127502  TXN_2025_00019695  2025-04-27  CUST_2022_00047463  PROD_001260   

                                 product_name     category  subcategory  \
0                Samsung Galaxy S6 16GB Black  Electronics  Smartphones   
1                OnePlus OnePlus 2 16GB White  Electronics  Smartphones   
2            Samsung Galaxy Note 5 64GB Black  Electronics  Smartphones   
3        Motorola Moto G (3rd Gen) 16GB Black  Electronics  Smartphones   
4                OnePlus OnePlus 2 16GB White  Electronics  Smartphones   
...                                       ...          ...          ...   
1127222            Realme iPad 4GB RAM Silver  Electronics      Tablets   
1127223     Samsung Galaxy S7 Edge 32GB Black  Electronics  Smartphones   
1127325                  Vivo Y20 128GB Black  Electronics  Smartphones   
1127494         OnePlus OnePlus 6 128GB White  Electronics  Smartphones   
1127502       Samsung Galaxy S24+ 256GB Black  Electronics  Smartphones   

            brand  original_price_inr  discount_percent  ...  order_month  \
0         Samsung           123614.29             27.91  ...            1   
1         OnePlus            54731.86              0.00  ...            1   
2         Samsung            97644.25             46.93  ...            1   
3        Motorola            21947.26              0.00  ...            1   
4         OnePlus            54731.86              0.00  ...            1   
...           ...                 ...               ...  ...          ...   
1127222    Realme           133854.95              0.00  ...           12   
1127223   Samsung            99038.79              0.00  ...           12   
1127325      Vivo            18508.73             18.74  ...            4   
1127494   OnePlus            40801.60             28.62  ...           12   
1127502   Samsung            50421.29             18.26  ...            4   

         order_year  order_quarter  product_weight_kg  is_prime_eligible  \
0              2015              1               0.19               True   
1              2015              1               0.20               True   
2              2015              1               0.17               True   
3              2015              1               0.22               True   
4              2015              1               0.20               True   
...             ...            ...                ...                ...   
1127222        2025              4               0.51               True   
1127223        2025              4               0.24               True   
1127325        2025              2               0.19               True   
1127494        2025              4               0.24               True   
1127502        2025              2               0.23               True   

        product_rating order_date_cleaned calc_discounted_price  \
0                  4.7         2015-01-25              89113.54   
1                  4.1         2015-01-05              54731.86   
2                  3.3         2015-01-24              51819.80   
3                  3.5         2015-01-28              21947.26   
4                  4.1         2015-0

In [205]:
df_sales['payment_method'].value_counts()

payment_method
UPI            382363
COD            321275
Credit Card    171405
Debit Card     139492
Net Banking     64623
Wallet          22678
BNPL            20198
Name: count, dtype: int64

In [206]:
df_sales.to_csv(r"C:\GUVI Projects\Sales Analytics - Amazon India\Data\Sales.csv")

In [3]:
import pandas as pd
import numpy as np
df_sales = pd.read_csv(r"C:\GUVI Projects\Sales Analytics - Amazon India\Data\Sales.csv")
df_sales.shape

C:\Users\raagh\AppData\Local\Temp\ipykernel_14740\1038566198.py:3: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  df_sales = pd.read_csv(r"C:\GUVI Projects\Sales Analytics - Amazon India\Data\Sales.csv")


(1122034, 39)

#Store data into database

In [4]:
import mysql.connector
connection = mysql.connector.connect(
    host = 'localhost',
    user = 'root',
    password = '12345678'
)

cursor = connection.cursor()


In [5]:
cursor.execute("USE Sales")

In [ ]:
#Table for customer spending tier
sql_create_tier = """CREATE TABLE IF NOT EXISTS Customer_SpendingTier (
                    Id INT AUTO_INCREMENT PRIMARY KEY,
                    Tier VARCHAR(100))"""

cursor.execute(sql_create_tier)
connection.commit()

In [43]:
#Populate table customer spending tier
sql_insert_tier = r"INSERT INTO Customer_SpendingTier(Tier) VALUES (%s)"
unique_tier = df_sales['customer_spending_tier'].unique().tolist()
data = [(item,) for item in unique_tier]

cursor.executemany(sql_insert_tier, data)
connection.commit()

In [ ]:
#Table for customer tier
sql_create_ctier = """CREATE TABLE IF NOT EXISTS Customer_Tier (
                    Id INT AUTO_INCREMENT PRIMARY KEY,
                    Tier VARCHAR(100))"""

cursor.execute(sql_create_ctier)
connection.commit()

In [240]:
df_sales['customer_tier'].unique()

array(['Metro', 'Rural', 'Tier2', 'Tier1'], dtype=object)

In [241]:
#Populate table customer tier
sql_insert_ctier = r"INSERT INTO Customer_Tier(Tier) VALUES (%s)"
unique_ctier = df_sales['customer_tier'].unique().tolist()
data = [(item,) for item in unique_ctier]

cursor.executemany(sql_insert_ctier, data)
connection.commit()

In [135]:
#Table for customer age group
sql_create_age = """CREATE TABLE IF NOT EXISTS Customer_age_group (
                    Id INT AUTO_INCREMENT PRIMARY KEY,
                    Age_Group VARCHAR(20))"""

cursor.execute(sql_create_age)
connection.commit()

In [139]:
df_sales['customer_age_group'].dropna().sort_values().unique().tolist()

['18-25', '26-35', '36-45', '46-55', '55+']

In [140]:
#Populate table customer age group
sql_insert_age = r"INSERT INTO Customer_age_group(Age_Group) VALUES (%s)"
unique_age = df_sales['customer_age_group'].dropna().sort_values().unique().tolist()
data = [(item,) for item in unique_age]

cursor.executemany(sql_insert_age, data)
connection.commit()

In [147]:
#Table for Festive sale
sql_create_fest = """CREATE TABLE IF NOT EXISTS Festive_Sale (
                    Id INT AUTO_INCREMENT PRIMARY KEY,
                    Name VARCHAR(100))"""

cursor.execute(sql_create_fest)
connection.commit()

In [150]:
df_sales['festival_name'].dropna().unique().tolist()

['Republic Day Sale',
 'Valentine Sale',
 'Holi Festival',
 'Summer Sale',
 'Back to School',
 'Prime Day',
 'Amazon Great Indian Festival',
 'Diwali Sale']

In [148]:
#Populate table Festive sale
sql_insert_fest = r"INSERT INTO Festive_Sale(Name) VALUES (%s)"
unique_fest = df_sales['festival_name'].dropna().unique().tolist()
data = [(item,) for item in unique_fest]

cursor.executemany(sql_insert_fest, data)
connection.commit()

In [ ]:
#Table for delivery status
sql_create_del = """CREATE TABLE IF NOT EXISTS Delivery_Status (
                    Id INT AUTO_INCREMENT PRIMARY KEY,
                    Name VARCHAR(100))"""

cursor.execute(sql_create_del)
connection.commit()

In [154]:
df_sales['return_status'].unique().tolist()

['Delivered', 'Returned', 'Cancelled']

In [155]:
#Populate table delivery status
sql_insert_del = r"INSERT INTO Delivery_Status(Name) VALUES (%s)"
unique_del = df_sales['return_status'].dropna().unique().tolist()
data = [(item,) for item in unique_del]

cursor.executemany(sql_insert_del, data)
connection.commit()

In [156]:
#Table for state
sql_create_state = """CREATE TABLE IF NOT EXISTS State (
                    Id INT AUTO_INCREMENT PRIMARY KEY,
                    Name VARCHAR(100))"""

cursor.execute(sql_create_state)
connection.commit()

In [160]:
df_sales['customer_state'].sort_values().unique().tolist()

['Andhra Pradesh',
 'Bihar',
 'Delhi',
 'Gujarat',
 'Karnataka',
 'Kerala',
 'Madhya Pradesh',
 'Maharashtra',
 'Odisha',
 'Punjab',
 'Rajasthan',
 'Tamil Nadu',
 'Telangana',
 'Uttar Pradesh',
 'West Bengal']

In [161]:
#Populate table state
sql_insert_state = r"INSERT INTO State(Name) VALUES (%s)"
unique_state = df_sales['customer_state'].sort_values().unique().tolist()
data = [(item,) for item in unique_state]

cursor.executemany(sql_insert_state, data)
connection.commit()

In [162]:
#Table for city
sql_create_city = """CREATE TABLE IF NOT EXISTS City (
                    Id INT AUTO_INCREMENT PRIMARY KEY,
                    State_Id INT,
                    Name VARCHAR(100),
                    FOREIGN KEY(State_Id) REFERENCES State(Id))"""

cursor.execute(sql_create_city)
connection.commit()

In [166]:
unique_city = df_sales.drop_duplicates(subset=['customer_state', 'customer_city'])
unique_city = unique_city[['customer_state', 'customer_city']]

df_state = pd.read_sql("SELECT Id as State_Id, Name FROM State", connection)
unique_city = unique_city.merge(df_state, left_on='customer_state', right_on='Name' ).drop(['Name','customer_state'], axis=1)
unique_city.sort_values('customer_city')

C:\Users\raagh\AppData\Local\Temp\ipykernel_4904\2276160013.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_state = pd.read_sql("SELECT Id as State_Id, Name FROM State", connection)


customer_city  State_Id
8       Ahmedabad         4
19        Aligarh        14
1       Allahabad        14
26       Bareilly        14
9       Bengaluru         5
7     Bhubaneswar         9
17     Chandigarh        10
12        Chennai        12
23     Coimbatore        12
15      Gorakhpur        14
18      Hyderabad        13
20         Indore         7
6          Jaipur        11
16         Kanpur        14
11          Kochi         6
2         Kolkata        15
5         Lucknow        14
3        Ludhiana        10
22         Meerut        14
27      Moradabad        14
0          Mumbai         8
13         Nagpur         8
4       New Delhi         3
21          Patna         2
10           Pune         8
25     Saharanpur        14
28          Surat         4
24       Vadodara         4
29       Varanasi        14
14  Visakhapatnam         1

In [167]:
#Populate table city
sql_insert_city = r"INSERT INTO City(Name,State_Id) VALUES (%s, %s)"
data = list(unique_city.sort_values('customer_city').itertuples(index=False, name=None))

cursor.executemany(sql_insert_city, data)
connection.commit()

In [210]:
#Table for Delivery type
sql_create_delType = """CREATE TABLE IF NOT EXISTS Delivery_Type (
                    Id INT AUTO_INCREMENT PRIMARY KEY,
                    Name VARCHAR(100))"""

cursor.execute(sql_create_delType)
connection.commit()

In [209]:
df_sales['delivery_type'].value_counts()

delivery_type
Standard    780112
Same Day    213696
Express     128226
Name: count, dtype: int64

In [211]:
#Populate table Delivery type
sql_insert_delType = r"INSERT INTO Delivery_Type(Name) VALUES (%s)"
unique_delType = df_sales['delivery_type'].unique().tolist()
data = [(item,) for item in unique_delType]

cursor.executemany(sql_insert_delType, data)
connection.commit()

In [215]:
#Table for Payment method
sql_create_pay = """CREATE TABLE IF NOT EXISTS Payment_Method (
                    Id INT AUTO_INCREMENT PRIMARY KEY,
                    Name VARCHAR(100))"""

cursor.execute(sql_create_pay)
connection.commit()

In [214]:
df_sales['payment_method'].unique()

array(['COD', 'Debit Card', 'Credit Card', 'Net Banking', 'UPI', 'Wallet',
       'BNPL'], dtype=object)

In [216]:
#Populate table payment method
sql_insert_pay = r"INSERT INTO Payment_Method(Name) VALUES (%s)"
unique_pay = df_sales['payment_method'].unique().tolist()
data = [(item,) for item in unique_pay]

cursor.executemany(sql_insert_pay, data)
connection.commit()

In [227]:
df_sales[['product_id','product_rating','product_weight_kg']].drop_duplicates()

product_id  product_rating  product_weight_kg
0        PROD_000021             4.7               0.19
1        PROD_000055             4.1               0.20
2        PROD_000039             3.3               0.17
3        PROD_000085             3.5               0.22
5        PROD_001665             3.6               1.39
...              ...             ...                ...
1066542  PROD_001397             4.6               0.25
1074698  PROD_001512             3.7               0.16
1077218  PROD_001486             4.5               0.18
1085722  PROD_001409             3.6               0.19
1086267  PROD_001505             3.5               0.15

[2004 rows x 3 columns]

In [320]:
sql_create_cust = """CREATE TABLE IF NOT EXISTS Customer(
                    Id VARCHAR(20) PRIMARY KEY,
                    City_Id INT,
                    State_Id INT,
                    Customer_Tier_Id INT,
                    Customer_SpendingTier_Id INT,
                    Customer_Age_Group_Id INT,
                    Is_Prime_Member BOOL,
                    FOREIGN KEY(City_Id) REFERENCES City(Id),
                    FOREIGN KEY(State_Id) REFERENCES State(Id),
                    FOREIGN KEY(Customer_Tier_Id) REFERENCES Customer_Tier(Id),
                    FOREIGN KEY(Customer_SpendingTier_Id) REFERENCES Customer_SpendingTier(Id),
                    FOREIGN KEY(Customer_Age_Group_Id) REFERENCES Customer_Age_Group(Id)
                    )"""
cursor.execute(sql_create_cust)
connection.commit()

In [321]:
df_cust = df_sales[['customer_id','customer_city', 'customer_state', 'customer_tier', 'customer_spending_tier', 'customer_age_group','is_prime_member']].drop_duplicates(subset='customer_id', keep='last')
df_cust.shape


(354969, 7)

In [322]:
# Load all refernce table data into dataframe 
df_city = pd.read_sql("SELECT c.Id as City_Id, c.State_Id, s.Name as State_Name, c.Name as City_Name FROM City c INNER JOIN State s ON c.State_Id = s.Id", connection)
df_cTier = pd.read_sql("SELECT Id as Customer_Tier_Id, Tier FROM Customer_Tier", connection)
df_sTier = pd.read_sql("SELECT Id as Customer_SpendingTier_Id, Tier FROM Customer_SpendingTier", connection)
df_age = pd.read_sql("SELECT Id as Customer_Age_Group_Id, Age_Group FROM Customer_Age_Group", connection)

#Merge tables to get ref Id
merged_tables = df_cust.merge(df_city, how='left',left_on=['customer_state','customer_city'], right_on=['State_Name','City_Name'] ).drop(['State_Name','City_Name'], axis=1)
merged_tables = merged_tables.merge(df_cTier, how='left',left_on='customer_tier', right_on='Tier' ).drop('Tier', axis=1)
merged_tables = merged_tables.merge(df_sTier, how='left',left_on='customer_spending_tier', right_on='Tier' ).drop('Tier', axis=1)
merged_tables = merged_tables.merge(df_age, how='left',left_on='customer_age_group', right_on='Age_Group' ).drop('Age_Group', axis=1)
merged_tables.shape

C:\Users\raagh\AppData\Local\Temp\ipykernel_4904\1472417026.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_city = pd.read_sql("SELECT c.Id as City_Id, c.State_Id, s.Name as State_Name, c.Name as City_Name FROM City c INNER JOIN State s ON c.State_Id = s.Id", connection)
C:\Users\raagh\AppData\Local\Temp\ipykernel_4904\1472417026.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_cTier = pd.read_sql("SELECT Id as Customer_Tier_Id, Tier FROM Customer_Tier", connection)
C:\Users\raagh\AppData\Local\Temp\ipykernel_4904\1472417026.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other 

(354969, 12)

In [323]:
merged_tables.columns
req_cols = ['customer_id', 'City_Id', 'State_Id', 'Customer_Tier_Id', 'Customer_SpendingTier_Id',
       'Customer_Age_Group_Id', 'is_prime_member']
merged_tables = merged_tables[req_cols].fillna(np.nan).replace([np.nan], [None])

In [298]:
print(len(req_cols))
','.join(['%s']*len(req_cols))

7


'%s,%s,%s,%s,%s,%s,%s'

In [324]:
#populate table Customer
sql_insert_cust = f"INSERT INTO Customer VALUES({','.join(['%s']*len(req_cols))})"
data = list(merged_tables.itertuples(index=False, name=None))

cursor.executemany(sql_insert_cust, data)
connection.commit()

In [325]:
df_sales[['festival_name', 'is_festival_sale']].drop_duplicates()

festival_name  is_festival_sale
0                 Republic Day Sale              True
1                               NaN              True
4                               NaN             False
2403                 Valentine Sale              True
4503                  Holi Festival              True
9305                    Summer Sale              True
12000                Back to School              True
14413                     Prime Day              True
19502  Amazon Great Indian Festival              True
22200                   Diwali Sale              True

In [6]:
#Create table Transaction
sql_create_tran = """CREATE TABLE IF NOT EXISTS Transaction(
                        Id VARCHAR(20) PRIMARY KEY,
                        Order_Date DATETIME,
                        Customer_Id	VARCHAR(20),
                        Product_Id VARCHAR(25),
                        Original_Price FLOAT,
                        Discount FLOAT,
                        Discounted_Price FLOAT,
                        Quantity INT, 
                        SubTotal FLOAT, 
                        Delivery_Charges FLOAT, 
                        Final_Amount FLOAT,
                        Payment_Method_Id INT,
                        Delivery_Days INT, 
                        Delivery_Type_Id INT, 
                        Is_Festival_Sale BOOL,
                        Festive_Sale_Id INT, 
                        Customer_Rating FlOAT, 
                        Delivery_Status_Id INT,
                        Order_Quarter INT,
                        Is_Prime_Eligible BOOL,
                        FOREIGN KEY(Product_Id) REFERENCES Product(Id),
                        FOREIGN KEY(Customer_Id) REFERENCES Customer(Id),
                        FOREIGN KEY(Payment_Method_Id) REFERENCES Payment_Method(Id),
                        FOREIGN KEY(Delivery_Type_Id) REFERENCES Delivery_Type(Id),
                        FOREIGN KEY(Festive_Sale_Id) REFERENCES Festive_Sale(Id),
                        FOREIGN KEY(Delivery_Status_Id) REFERENCES Delivery_Status(Id)
                    )"""

cursor.execute(sql_create_tran)
connection.commit()

In [7]:
df_sales.columns
df_trans = df_sales[['transaction_id', 'order_date_cleaned', 'customer_id', 'product_id',
          'calc_original_price', 'discount_percent',  'calc_discounted_price2',
        'quantity', 'subtotal_inr', 'delivery_charges', 'final_amount_inr','payment_method',
       'delivery_days', 'delivery_type','is_festival_sale',
       'festival_name', 'customer_rating', 'return_status',
       'order_quarter', 'is_prime_eligible']]
df_trans.shape

(1122034, 20)

In [8]:
# Load all refernce table data into dataframe 
df_cust = pd.read_sql("SELECT Id FROM Customer", connection)
df_pay = pd.read_sql("SELECT Id as Payment_Method_Id, Name FROM Payment_Method", connection)
df_del = pd.read_sql("SELECT Id as Delivery_Type_Id, Name FROM Delivery_Type", connection)
df_fest = pd.read_sql("SELECT Id as Festive_Sale_Id, Name FROM Festive_Sale", connection)
df_status = pd.read_sql("SELECT Id as Delivery_Status_Id, Name FROM Delivery_Status", connection)

#Merge tables to get ref Id
merged_tables = df_trans.merge(df_cust, left_on='customer_id', right_on='Id' ).drop('Id', axis=1)
merged_tables = merged_tables.merge(df_pay, how='left', left_on='payment_method', right_on='Name' ).drop('Name', axis=1)
merged_tables = merged_tables.merge(df_del, how='left', left_on='delivery_type', right_on='Name' ).drop('Name', axis=1)
merged_tables = merged_tables.merge(df_fest, how='left', left_on='festival_name', right_on='Name' ).drop('Name', axis=1)
merged_tables = merged_tables.merge(df_status, how='left', left_on='return_status', right_on='Name' ).drop('Name', axis=1)
merged_tables

C:\Users\raagh\AppData\Local\Temp\ipykernel_14740\35678421.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_cust = pd.read_sql("SELECT Id FROM Customer", connection)
C:\Users\raagh\AppData\Local\Temp\ipykernel_14740\35678421.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_pay = pd.read_sql("SELECT Id as Payment_Method_Id, Name FROM Payment_Method", connection)
C:\Users\raagh\AppData\Local\Temp\ipykernel_14740\35678421.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_del = pd.read_sql("SELECT Id as Deliver

transaction_id order_date_cleaned         customer_id  \
0        TXN_2015_00000001         2015-01-25  CUST_2015_00003884   
1        TXN_2015_00000002         2015-01-05  CUST_2015_00011709   
2        TXN_2015_00000003         2015-01-24  CUST_2015_00004782   
3        TXN_2015_00000004         2015-01-28  CUST_2015_00008105   
4        TXN_2015_00000005         2015-01-31  CUST_2015_00002955   
...                    ...                ...                 ...   
1122029  TXN_2025_00076999         2025-12-12  CUST_2025_00019394   
1122030  TXN_2025_00077000         2025-12-24  CUST_2021_00014781   
1122031  TXN_2025_00022234         2025-04-27  CUST_2015_00004317   
1122032  TXN_2025_00067929         2025-12-13  CUST_2024_00037053   
1122033  TXN_2025_00019695         2025-04-27  CUST_2022_00047463   

          product_id  calc_original_price  discount_percent  \
0        PROD_000021            123614.29             27.91   
1        PROD_000055             54731.86              0.00   
2        PROD_000039             97644.25             46.93   
3        PROD_000085             21947.26              0.00   
4        PROD_000055             54731.86              0.00   
...              ...                  ...               ...   
1122029  PROD_001739            133854.95              0.00   
1122030  PROD_000133             99038.79              0.00   
1122031  PROD_000771             18508.73             18.74   
1122032  PROD_000388             40801.60             28.62   
1122033  PROD_001260             50421.29             18.26   

         calc_discounted_price2  quantity  subtotal_inr  delivery_charges  \
0                      89113.54         1      89112.17               0.0   
1                      54731.86         1      54731.86               0.0   
2                      51819.80         2     103636.52               NaN   
3                      21947.26         1      21947.26               0.0   
4                      54731.86         2     109463.72               0.0   
...                         ...       ...           ...               ...   
1122029               133854.95         1     133854.95               0.0   
1122030                99038.79         1      99038.79               0.0   
1122031                15040.19         1      15040.64               0.0   
1122032                29124.18         1      29124.51               0.0   
1122033                41214.36         1      41215.32               0.0   

         ...  is_festival_sale      festival_name  customer_rating  \
0        ...              True  Republic Day Sale              5.0   
1        ...              True                NaN              4.5   
2        ...              True  Republic Day Sale              NaN   
3        ...              True                NaN              3.0   
4        ...             False                NaN              4.0   
...      ...               ...                ...              ...   
1122029  ...             False                NaN              5.0   
1122030  ...              True                NaN              3.5   
1122031  ...             False                NaN              3.5   
1122032  ...              True                NaN              NaN   
1122033  ...              True                NaN              NaN   

        return_status  order_quarter is_prime_eligible  Payment_Method_Id  \
0           Delivered              1              True                  1   
1           Delivered              1              True                  1   
2           Delivered              1              True                  1   
3           Delivered              1              True                  1   
4           Delivered              1              True                  1   
...               ...            ...               ...                ...   
1122029     Delivered              4              True                  5   
1122030     Delivered              4        

In [9]:
merged_tables.columns
req_cols = ['transaction_id', 'order_date_cleaned', 'customer_id', 'product_id',
       'calc_original_price', 'discount_percent', 'calc_discounted_price2',
       'quantity', 'subtotal_inr', 'delivery_charges', 'final_amount_inr',
       'Payment_Method_Id', 'delivery_days', 'Delivery_Type_Id', 'is_festival_sale',
       'Festive_Sale_Id', 'customer_rating', 'Delivery_Status_Id', 'order_quarter',
       'is_prime_eligible']
merged_tables = merged_tables[req_cols].fillna(np.nan).replace([np.nan], [None])
merged_tables.drop_duplicates(subset='transaction_id', keep='first', inplace=True)
merged_tables.shape

(1121999, 20)

In [330]:
print(len(req_cols))
','.join(['%s']*len(req_cols))

20


'%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s'

In [10]:
#populate table transaction
chunk_size = 40000
sql_insert_trans = f"INSERT INTO Transaction VALUES({','.join(['%s']*len(req_cols))})"
for chunk in np.array_split(merged_tables, len(merged_tables) // chunk_size + 1):
    data = [tuple(x) for x in chunk.to_numpy()]
    cursor.executemany(sql_insert_trans, data)
    connection.commit()

# sql_insert_trans = f"INSERT INTO Transaction VALUES({','.join(['%s']*len(req_cols))})"
# data = list(merged_tables.itertuples(index=False, name=None))

# cursor.executemany(sql_insert_trans, data)
# connection.commit()

c:\Users\raagh\AppData\Local\Programs\Python\Python314\Lib\site-packages\numpy\_core\fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
